# Regressão Linear - Implantação

Este componente realiza predições usando um modelo de Regressão Linear usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para ajustes de modelos, pré-processamento de dados, seleção e avaliação de modelos, além de outras funcionalidades.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/linear-regression.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.columns_to_filter = artifacts["columns_to_filter"]
        self.new_columns = artifacts["new_columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]

    def class_names(self):
        column_names = np.concatenate(
            (self.columns_to_filter, [self.new_columns])
        )
        return column_names.tolist()

    def predict(self, X, feature_names, meta=None):
        
        df = pd.DataFrame(X)
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)[self.columns_to_filter]
            X = df.to_numpy()

        # Realiza a predição
        y_pred = self.pipeline.predict(X)

        # adiciona a predição ao conjunto de dados
        df[self.new_columns] = y_pred

        return df.to_numpy()